<a href="https://colab.research.google.com/github/Nidhinbc97/ML-AI/blob/main/Assignment_07_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_colwidth',None)

In [2]:
data = pd.read_csv('/content/tweets.csv')

In [3]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [4]:
data.isna().sum()

id       0
label    0
tweet    0
dtype: int64

In [5]:
data.describe()

,id,label
count,7920.000000,7920.000000
mean,3960.500000,0.255808
std,2286.451399,0.436342
min,1.000000,0.000000
25%,1980.750000,0.000000
50%,3960.500000,0.000000
75%,5940.250000,1.000000
max,7920.000000,1.000000


In [8]:
data['label'].value_counts()

label
0    5894
1    2026
Name: count, dtype: int64

In [9]:
data['cleaned_text'] = data['tweet'].apply(lambda x:gensim.utils.simple_preprocess(x))
data.head()

,id,label,tweet,cleaned_text
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, https, goo, gl, mfqv, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias, http, instagram, com, yget, jc, jm]"
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect, http, fb, me, lsupcu]"
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home, http, instagr, am, li_]"
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [10]:
data.columns

Index(['id', 'label', 'tweet', 'cleaned_text'], dtype='object')

In [11]:
x_train,x_test,y_train,y_test=train_test_split(data['cleaned_text'],data['label'],test_size=0.3)

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5544,)
(2376,)
(5544,)
(2376,)


In [13]:
w2v_model = gensim.models.Word2Vec(x_train,min_count=2)

In [14]:
words = w2v_model.wv.index_to_key
len(words)

5421

In [15]:
x_train_vec = []
x_test_vec = []

for rvw in x_train:
  vec = []
  for word in rvw:
    if word in words:
      vec.append(w2v_model.wv['word'])

  x_train_vec.append(np.array(vec))

for rvw in x_test:
  vec = []
  for word in rvw:
    if word in words:
      vec.append(w2v_model.wv['word'])

  x_test_vec.append(np.array(vec))

In [16]:
x_train_vec_avg =[]
x_test_vec_avg = []

def cal_avg_vec(v):
  if v.size:
    return v.mean(axis = 0)
  else:
    np.zeros(100,dtype=float)

for v in x_train_vec:
  x_train_vec_avg.append(cal_avg_vec(v))

for v in x_test_vec:
  x_test_vec_avg.append(cal_avg_vec(v))

In [17]:
x_train_vec_avg = np.array(x_train_vec_avg)
x_test_vec_avg = np.array(x_test_vec_avg)

In [18]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train_vec_avg,y_train)

RandomForestClassifier()

In [19]:
y_pred = clf.predict(x_test_vec_avg)

In [20]:
accuracy_score(y_test,y_pred)

0.7293771043771043